In [1]:

# CS0155 Term Project - Spring 2017

## Section 1 -- Team Membership:
----
* **Leader (submits assignment):** Kekaimalu Chin
* **Email**: kgc7@pitt.edu
* **PeopleSoftID**: 3873115
----
* **Team Member #1 Erika Carpio 
* **Email**: erc72@pitt.edu  
* **PeopleSoftID**: 3910236
----
* **Team Member #2 Michael Rundell  
* **Email**: mdr48@pitt.edu  
* **PeopleSoftID**: 3799925   
----
By submitting this project we declare that
* the work in the project was done equally among all team members,  
* we did not share our work with others, and   
* we did not receive material from others.

SyntaxError: invalid syntax (<ipython-input-1-fd0404b626c6>, line 5)

In [ ]:
#PLEASE RUN THIS SECTION FIRST BECAUSE IT SETS UP THE CONNECTION AND THE IMPORTS FOR VISUALIZATIONS


# Import SQLite Python module
import sqlite3 as lite

# Connect to real-estate.db sqlite database
con = lite.connect('real-estate.db')

# Establish cursor with database
cur = con.cursor()


#all the imports used for the visualization: 
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import pandas.io.sql as psql


# Sample query 
myq = """
select count(*) as co 
from real_estate
"""
cur.execute(myq)
cur.fetchall()


In [ ]:
# Verify View used in Q8 is here
myq = """
select *
from most_expensive
"""
cur.execute(myq)
cur.fetchall()

## Section 2 -- Queries

In [ ]:
# Q1
q1 = """
select propertyzip, max(SALEPRICE) 
from 'real_estate'
group by propertyzip
order by saleprice  DESC
"""
cur.execute(q1)
cur.fetchall()

In [ ]:
# Q2
highestView= """
CREATE VIEW highestView AS
select propertyzip as zip, max(SALEPRICE) as max
from 'real_estate'
group by propertyzip
order by saleprice DESC
"""

q2 = """
SELECT real_estate.propertyzip, PROPERTYADDRESS, PROPERTYCITY, SALEPRICE
FROM
     'real_estate'
     INNER JOIN highestView ON real_estate.propertyzip = highestView.zip AND real_estate.saleprice = highestView.max
group by propertyaddress
order by real_estate.propertyzip ASC

"""
dropQuery ="""
DROP VIEW if exists highestView
"""
#used dropquery to make sure the view wasnt already created every time I run it. 
cur.execute(dropQuery)
cur.execute(highestView)
cur.execute(q2)
cur.fetchall()

In [ ]:
# Q3
q3 = """
select PARID, propertyzip, RECORDDATE, SALEDATE
from 'real_estate'
WHERE NOT RECORDDATE = "None" AND NOT SALEDATE = "None" 
AND CAST(SUBSTR(RECORDDATE,7,7) AS INT) >= (CAST(SUBSTR(SALEDATE,7,7) AS INT) + 2)
OR CAST(SUBSTR(RECORDDATE,7,7) AS INT) <= (CAST(SUBSTR(SALEDATE,7,7) AS INT) - 2)
"""
cur.execute(q3)
cur.fetchall()

In [ ]:
# Q4
date_errors = """
create view date_errors AS
select PARID, propertyzip, RECORDDATE, SALEDATE
from 'real_estate'
WHERE NOT RECORDDATE = "None" AND NOT SALEDATE = "None" 
AND CAST(SUBSTR(RECORDDATE,7,7) AS INT) >= (CAST(SUBSTR(SALEDATE,7,7) AS INT) + 2)
OR CAST(SUBSTR(RECORDDATE,7,7) AS INT) <= (CAST(SUBSTR(SALEDATE,7,7) AS INT) - 2)
"""

q4 = """
select propertyzip, count(*)
from date_errors
group by propertyzip
order by count(*) DESC
"""
dropQuery= """
DROP VIEW if exists date_errors
"""


#I used the drop query to drop the view every time I ran it, coulda just not run the date_errors query but i wanted to test that mostly. 
cur.execute(dropQuery)
cur.execute(date_errors)
cur.execute(q4)
cur.fetchall()

In [ ]:
# Q5
q5 = """
SELECT SUBSTR(SALEDATE,7,7) as date, count(SUBSTR(SALEDATE,7,7)) as count
FROM 'real_estate'
WHERE SCHOOLDESC = "City Of Pittsburgh" 
group by SUBSTR(SALEDATE,7,7)
order by SUBSTR(SALEDATE,7,7) ASC
"""
cur.execute(q5)
cur.fetchall()

In [ ]:
# Q6
q6 = """
select date, school, count(*)
from q5view
group by date
order by count(*) DESC
"""

q5view = """
create view q5view AS
SELECT SUBSTR(SALEDATE,7,7) as date, SCHOOLDESC as school
FROM 'real_estate'
"""
dropQuery= """
DROP VIEW if exists q5view
"""
#used dropQuery to make sure the view doesn't already exist
cur.execute(dropQuery)
cur.execute(q5view)
cur.execute(q6)
cur.fetchall()

In [ ]:
# Q7
q7 = """
 select SCHOOLDESC as school,AVG(FIREPLACES) as avgFirePlaces
 from real_estate 
 group by SCHOOLDESC 
 order by AVG(FIREPLACES) DESC 
 LIMIT 10;
"""

cur.execute(q7)
cur.fetchall()

In [ ]:
# Q8
q8 = """
SELECT count(fireDESC)
FROM avgfireplaces
INNER JOIN most_expensive ON most_expensive.expensiveDESC = avgfireplaces.fireDESC
"""
q7view = """
create view avgfireplaces as
 select SCHOOLDESC AS fireDESC, AVG(FIREPLACES) 
 from real_estate 
 group by SCHOOLDESC 
 order by AVG(FIREPLACES) DESC 
 LIMIT 10;

"""
q8givenView = """
create view most_expensive as
select SCHOOLDESC as expensiveDESC, avg(saleprice) as aprice
from real_estate
group by SCHOOLDESC
order by aprice DESC
LIMIT 10;
"""
dropQuery= """
DROP VIEW if exists most_expensive
"""
dropQuery2= """
DROP VIEW if exists avgfireplaces
"""

#again used the dropQuery and dropQuery2 to make sure the view doesnt alreayd exist
cur.execute(dropQuery)
cur.execute(dropQuery2)
cur.execute(q7view)
cur.execute(q8givenView)
cur.execute(q8)
cur.fetchall()

In [ ]:
# Q9

q9 = """
select schooldesc, avg(finishedlivingarea)
from real_estate
where NOT FINISHEDLIVINGAREA = "null" 
group by schooldesc
order by avg(finishedlivingarea) DESC
limit 10
"""

cur.execute(q9)
cur.fetchall()

In [ ]:
# Q10
q10 = """
select schooldesc, (avg(SALEPRICE)/avg(finishedlivingarea)) as averagepricepersqft
from real_estate
where NOT FINISHEDLIVINGAREA = "null" AND not SALEPRICE = "null"
group by schooldesc
order by averagepricepersqft DESC
limit 10
"""
cur.execute(q10)
cur.fetchall()

## Section 3 -- Visualization

In [ ]:
# Q5 visualization

df = pd.read_sql_query(q5,con)
#print (df) -used for testing, can unselect if you wanna see the data itself.

df.plot(x = "date", y = "count")


In [ ]:
# Q7 visualization
cur.execute(q7)


df = pd.read_sql_query(q7,con)

df.plot.bar(x="school",y="avgFirePlaces")

df.plot()
# create a plot using q7res here